# Remote sensing workshop on Microsoft's Planetary Computer
Introduction to geopandas, STAC, rasterio, numpy for images, scikit-image, ...

In [ ]:
# Planetary Computer, STAC
import pystac
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer

# Stats and vector data
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely

# Image processing
import rasterio as rio
import rasterio.plot, rasterio.features, rasterio.mask, rasterio.fill  # Not loaded by default
import skimage.morphology, skimage.measure

# Plotting
import matplotlib.pyplot as plt
import IPython.display
import ipywidgets
import leafmap

# For nice HTML outputs
import rich.table

## Environment

In [ ]:
# Shell commands - know your env !
!source /etc/os-release && echo $PRETTY_NAME && uname -sir

In [ ]:
# This one is quite verbose
#!env

In [ ]:
# We're running on a fresh system
!conda --version && python --version

In [ ]:
# If you need to check / export conda env (with packages versions)
#!conda env export > conda_env.yml

In [ ]:
# Data in /home/jovyan will persist across sessions, with just 15GB of storage.
# We still have 163Gb to work with in / ; we can use /tmp to write files if needed
!df -h

In [ ]:
plt.style.use('default')
folium_tile = "CartoDB Positron"

# Or if you're a darkmode addict...

#folium_tile = "CartoDB Dark Matter"
#plt.style.use('dark_background')

## GeoPandas
The reference lib to work with GIS vector data in Python

In [ ]:
# Load json features from WFS
wfs_communes = "https://wxs.ign.fr/topographie/geoportail/wfs?SERVICE=WFS&VERSION=2.0.0" \
               "&request=GetFeature&OUTPUTFORMAT=application/json&typename=BDTOPO_V3:commune&CQL_FILTER=code_insee_du_departement=33"
# If you ever want to cache the result of the HTTP request, you could use the request lib to fetch the result and store it

In [ ]:
# But here GeoPandas will handle the request for us.
# It can read various types of URLs, locally or remote, files or archives, database queries, ...
communes = gpd.read_file(wfs_communes)
communes.info()

In [ ]:
# Drop datetime columns because it's incompatible with folium
communes.drop(['date_creation', 'date_modification'], axis=1, inplace=True)


In [ ]:
# Explore GeoPandas dataset in interactive folium (leaflet) map
communes.explore(tiles=folium_tile)

In [ ]:
# Select one commune as AOI for our first analysis
area_of_interest = communes.loc[communes.code_insee == '33529']

In [ ]:
# Basic plot with lat/long axis labels
area_of_interest.plot()

In [ ]:
# Get bounding box of each polygon in a GeoDataFrame
area_of_interest.bounds

In [ ]:
# Make sure your layer and bbox is in lat/long formatting, instead of any projected coordinates
area_of_interest.crs

In [ ]:
# Get bounds of the first (and only) multipolygon in our GeoDataFrame
bbox = tuple(area_of_interest.bounds.values[0])
# Another way to get GeoDataFrame bounds (layer wide)
assert bbox == rio.features.bounds(area_of_interest)
bbox

In [ ]:
# We want to ignore islands. Let's break this multipolygon to extract only the terrestrial part
polygons = list(area_of_interest.geometry.values[0])
polygons

In [ ]:
# The terrestrial part is the biggest one
polygons = sorted(polygons, key=lambda poly: poly.area, reverse=True)
polygons[0]

In [ ]:
# Get bounding box of our simple polygon (this will be our Region Of Interest)
roi = polygons[0]
bbox = roi.bounds
bbox

## STAC
[Spatio Temporal Asset Catalog](https://stacspec.org/) is the latest standard to search and acess Earth Observation datasets.

In [ ]:
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# If you work outside of Jupyter, set the environment variable PC_SDK_SUBSCRIPTION_KEY or use python function:
# planetary_computer.settings.set_subscription_key(YOUR_API_KEY)

In [ ]:
# Initialize Planetary Computer catalog instance
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
# Found two dates before and after the fire within our Region Of Interest
time_of_interest = "2022-06-01/2022-09-30"

In [ ]:
# Searching for S2-L2 products within ROI and time range
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,  # or intersects=area_of_interest
    datetime=time_of_interest,
    sortby='datetime',  # By default the query returns latest first, here we sortby ascending
    query={"eo:cloud_cover": {"lt": 10}},  # cloud cover lower than 10%
)
items = search.item_collection()
print(f"Found {len(items)} items\n")

In [ ]:
# TODO: Explore STAC items
items

In [ ]:
# Convert items metadata to GeoDataFrame
gdf_items = gpd.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
gdf_items.head(1)

In [ ]:
# Convert timestamps to datetime objects if you need to plot a time serie
gdf_items["datetime"] = pd.to_datetime(gdf_items.datetime)
fig, ax = plt.subplots(figsize=(10,2))
plt.scatter(gdf_items.datetime, gdf_items["eo:cloud_cover"])
ax.set_ylabel("Cloud cover (%)")

In [ ]:
# PNG image preview
items[0].assets["rendered_preview"]

In [ ]:
# You can access a specific dataset in a STAC item using its URL (aka href)
items[0].assets["rendered_preview"].href

## Display existing PNG images
Use IPython core to display images that are already rendered

In [ ]:
# Build IPython image object from pre-rendered image url
IPython.display.Image(url=items[0].assets["rendered_preview"].href, width=800)

In [ ]:
# Here we create IPython objects that we can use with IPywidgets
images = {it.id: IPython.display.Image(url=it.assets["rendered_preview"].href, width=500) for it in items}
names = list(images.keys())
images

In [ ]:
# Interactive function to show images, parameter name must be same than in interact() arguments
def loadimg(i=0):
    IPython.display.display(images[names[i]])
    return names[i]

In [ ]:
# This is how you can create a widget. The first display will be slow, before the PNG file is cached
ipywidgets.interact(loadimg,  i=(0, len(images)-1))

In [ ]:
# Pick two dates before and after the fire (mainly between 2022/07/12 and 2022/07/29)
before, after = items[2], items[4]
before, after

In [ ]:
# Single STAC item
before

## Explore a dataset

In [ ]:
# Rich provides an easy way to build pretty HTML tables
table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in before.assets.items():
    table.add_row(asset_key, asset.title)
table

In [ ]:
# Use Leafmap to load a STAC layer (Near Infra Red data)
m = leafmap.Map()
m.add_stac_layer(collection="sentinel-2-l2a", item=after.id, assets=["B08"])
# Add a GeoDataFrame polygon layer on top of your leafmap
m.add_gdf(area_of_interest, style={'color': 'yellow'})
m

In [ ]:
# Load full false color (NIR) image with Leafmap
m = leafmap.Map()
m.add_stac_layer(
    collection="sentinel-2-l2a",
    name=before.datetime.strftime('%Y.%m.%d'),
    item=before.id,
    assets=["B08", "B04", "B03"]
)
m

In [ ]:
# If you just need to visualize data, you can compute index on the fly
# Here we add two NDVI layers, rescaled between -0.5 and 0.5 for better display
m = leafmap.Map()
m.add_stac_layer(
    collection="sentinel-2-l2a",
    name=after.datetime.strftime('%Y.%m.%d'),
    item=after.id,
    assets=["B04", "B03", "B02", "B08"],
    expression="(B08 - B04) / (B08 + B04)",
    rescale="-0.5,0.5",
    colormap_name="rdylgn",
)
m.add_stac_layer(
    collection="sentinel-2-l2a",
    name=before.datetime.strftime('%Y.%m.%d'),
    item=before.id,
    assets=["B04", "B03", "B02", "B08"],
    expression="(B08 - B04) / (B08 + B04)",
    rescale="-0.5,0.5",
    colormap_name="rdylgn",
)
# Use the layer switcher to see the difference (top right icon, then first tab)
m

In [ ]:
# Avoid loading too many plots and HTML maps if you don't want your notebook checkpoints to become huge
# Images and JSON geometries can really make your map slow (and here we're using high-res polygons from IGN's WFS)
# You can comment the "m" when you're done with a map so the rendered html output deleted from current notebok state

## Rasterio
The best library to ease the use of GDAL with Python.

In [ ]:
# Open a remote 8bit GeoTIFF dataset to fetch image geometry
with rio.open(before.assets["visual"].href) as ds:
    profile = ds.profile
    # Do not not read data for now
    #array = data.read()

ds

In [ ]:
# Question: what actually happened here ? Do you know the "with" statement ?

Rasterio (as GDAL) needs 3 things to preserve an image geometry:
* shape (bands, height, width)
* crs (coordinate reference system)
* affine transformation (see [related package](https://github.com/rasterio/affine) git to learn more)

In [ ]:
# We can use the same profile dict for different bands since this 3 things are constant
rich.print(profile)

In [ ]:
# Let's keep a reference to the CRS
crs = profile['crs']
crs

In [ ]:
# To avoid loading the full image, we need to build a window
# To do so we need a warped (i.e. projected) boundind box, and the source image transform
warped_aoi_bounds = rio.warp.transform_bounds("epsg:4326", crs, *bbox)
warped_aoi_bounds

In [ ]:
aoi_window = rio.windows.from_bounds(*warped_aoi_bounds, transform=profile["transform"])
aoi_window

In [ ]:
# Round pixel size and offsets: we can't slice any array using floating points
# This is handled automatically by rasterio when you call ds.read(window=window)
aoi_window = aoi_window.round_shape().round_offsets()
aoi_window

In [ ]:
# Extract 8bit image within ROI, get data as numpy arrays
with rio.open(before.assets["visual"].href) as ds:
    viz_before = ds.read(window=aoi_window)

In [ ]:
with rio.open(after.assets["visual"].href) as ds:
    viz_after = ds.read(window=aoi_window)

In [ ]:
# Bands, Latitude, Longitude
print("bands: {}, rows: {}, cols: {}\n".format(*viz_before.shape) + f"dtype: {viz_before.dtype}")

In [ ]:
# Visualize histogram of pixel values.
fig, ax = plt.subplots(figsize=(10,4))
rio.plot.show_hist(viz_before, bins=64, stacked=False, alpha=0.5, title="Histogram")
# The legend is not displayed properly, most likely a rasterio matplotlib bug in our bleeding edge jupyter env

In [ ]:
# Question: How do you explain the last bins ?

## Numpy array manipulations

In [ ]:
# Matplotlib, PIL or skimage are expecting Cols,Rows,Bands order
np.moveaxis(viz_before, 0, 2).shape  # Move axis at index 0 to index 2

In [ ]:
# There a many ways to change axis order
np.moveaxis(viz_before, 0, 2).shape \
== np.transpose(viz_before, axes=[1, 2, 0]).shape \
== np.rollaxis(viz_before, 0, 3).shape \
== rio.plot.reshape_as_image(viz_before).shape

In [ ]:
# Plot two images side by side, matplotlib axis with pixel coords
fig, axes = plt.subplots(1,2, figsize=(12,16), sharey=True, layout="tight")
axes[0].imshow(rio.plot.reshape_as_image(viz_before))
axes[1].imshow(rio.plot.reshape_as_image(viz_after))
# Using imshow instead of basic plots ensure the width / height ratio is preserved

In [ ]:
# TODO: reproduce the plot above, but zoom into the burnt area using only numpy slicing

In [ ]:
# Question: how does Band 12 differs from optical (RGB) bands ?
before.assets["B12"]

In [ ]:
# Load 16bit data as numpy array, using rasterio to open remote COG file
def rio_stack(
    item: pystac.Item,
    band_list: list,
    window: rio.windows.Window = None
):
    """Load and stack different bands in a STAC item, return stacked array and geometry profile"""
    array_list = []
    # Collect arrays
    for band_id in band_list:
        url = item.assets[band_id].href
        with rasterio.open(url) as ds:
            profile = ds.profile
            data = ds.read(1, window=window)  # read(1) returns 2D array
            array_list.append(data)
    # Update metadata dict
    profile["count"] = len(array_list)
    if window is not None:
        # Update image geometry
        profile["transform"] = rio.windows.transform(aoi_window, profile["transform"])
        # ! Width and height are integer since we used aoi_window.round_shape() !
        profile["width"], profile["height"] = aoi_window.width, aoi_window.height

    # N.B. this function is valid if only if all bands have the same geometry
    return np.stack(array_list), profile

In [ ]:
# Here we also reorder BGR bands to RGB
array_before, profile = rio_stack(before, ["B04", "B03", "B02", "B08"], aoi_window)
# Make sure to check the profile dict
rich.print(profile)

In [ ]:
array_before.max()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
rio.plot.show_hist(array_before, bins=128, stacked=False, alpha=0.5, title="Histogram")

In [ ]:
# Write raster data (always make sure your profile dict is correct)
img_path = before.id + "_RGB_NIR.tif"
# Open in "write" mode, here you may pass other params for GDAL as kwargs (after **profile)
with rasterio.open(img_path, "w", **profile) as w:
    w.write(array_before)

In [ ]:
# TODO: use numpy to compute min, max, mean and std statistics for each band. 
# Avoid the for loop: it is possible with a single function call for each stat...
# You'll need to reshape the array first (or explore the docs for other options)

You should find this:
```
Min:  [0 0 0 0]
Max:  [17344 18032 10992 16624]
Mean [1590.0115479  1666.32281737 1440.60727583 2978.07257113]
Std [ 774.51116523  619.39048455  476.28939452 1337.2577833 ]
```

In [ ]:
# In this array, zero is used as nodata value.
# TODO: use numpy to compute nonzero minimum for each band, as in the previous cell
# Try to use the numpy.where and numpy.nanmin functions (and / or a masked array, see the next chapter).  
# Skip this if you don't know the np.where function (or see the docs to find another way).

In [ ]:
# Use slicing to get first bands
array_3b = array_before[:3]
array_3b.shape

In [ ]:
# Wooops
plt.imshow(np.moveaxis(array_3b, 0, 2))

In [ ]:
# Question: can you explain what's the problem here ?

In [ ]:
# If you ever need to build a rendered (encoded) image object from a numpy array (usable with IPython.display.Image), you can use PIL
import PIL
def pil_image_from_array(array, target_width=None):
    """Build PIL image object from a numpy array, optional image transform to target width"""
    # Another way to change bands order
    array = np.transpose(array, axes=[1, 2, 0])
    img = PIL.Image.fromarray(array)
    if not target_width:
        return img
    w = img.size[0]
    h = img.size[1]
    aspect = w / h
    target_height = (int)(target_width / aspect)
    return img.resize((target_width, target_height), PIL.Image.Resampling.BILINEAR)

In [ ]:
#pil_image_from_array(viz_after, target_width=1200)

## Masked numpy arrays
Most of the time, a raster dataset come with one or several masks (may be one per band) that will help you locate NaNs or pixels out of the projected grid.  
Dealing with masks / nodata can become a nightmare, we will only cover the basis of what is a mask, at the numpy level.  
See [rasterio docs](https://rasterio.readthedocs.io/en/latest/topics/masks.html) for more infos.

In [ ]:
with rasterio.open(before.assets['B08'].href) as ds:
    profile = ds.profile
    mask = ds.read_masks(window=aoi_window)
    data = ds.read(window=aoi_window)

In [ ]:
# Here data is not masked
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(data[0])

In [ ]:
assert mask.shape == data.shape
# A rasterio mask is encoded with 8bit, 255 for invalid data, else 0
mask

In [ ]:
# Check if mask is empty
mask.astype(bool).all()

In [ ]:
# Let's build a numpy masked array. We need to invert it since GDAL and Numpy aren't using the same convention
mask = ~mask.astype(bool)  # ~ operator (bitwise not) is equivalent to np.invert
mask

In [ ]:
# Build a mask, we decide to fill missing values with the dataset declared nodata value (e.g. zero)
masked_data = np.ma.masked_array(data, mask, fill_value=profile['nodata'])

In [ ]:
# A masked array is composed of 2 arrays of equal dimension : one with data, one with boolean mask
masked_data

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(masked_data[0])

In [ ]:
# The "base" array is an array with filled values where mask is True
print(f"{masked_data.min()=}, {masked_data.base.min()=}")

In [ ]:
# You can use vector data to mask pixels outside of ROI
def rio_clip_image(img_url: str, roi: gpd.GeoDataFrame):
    """OPen image and extract ROI, mask values outside of polygon(s)"""
    with rasterio.open(img_url, "r") as src:
        # Convert vector data CRS to the same as our image
        extent = roi.to_crs(src.profile['crs']).geometry
        out_img, out_transform = rasterio.mask.mask(
            src, extent, filled=False, crop=True  # crop=True ensure we do not return the full image extent
        )
        out_meta = src.meta
        out_meta.pop("nodata")

        out_meta.update(
            {
                "height": out_img.shape[1],
                "width": out_img.shape[2],
                "transform": out_transform,
            }
        )

        return out_img, out_meta

In [ ]:
masked_data, meta = rio_clip_image(before.assets['B08'].href, area_of_interest)

In [ ]:
# Here we can see our ROI mask is merged with the dataset nodata mask
plt.subplots(figsize=(10,10))
plt.imshow(masked_data[0], cmap="magma", interpolation="bilinear")

In [ ]:
# TODO: use the skimage.morphology.remove_small_objects to clean small blobs in your boolean mask (masked_data.mask).
# You need to create a new masked array (using source masked_data.base) and cleaned boolean array.
# Plot the mask before and after the cleaning.

## Computations

In [ ]:
red, green, blue, nir = array_before

In [ ]:
ndvi = (nir - red) / (nir + red)

In [ ]:
# Max(NDVI) should always be 1. Something went wrong here..
# Question: can you explain why ?
np.nanmax(ndvi)

In [ ]:
red, green, blue, nir = array_before.astype(np.float32)
ndvi = (nir - red) / (nir + red)
# Yes, this is better
assert np.nanmax(ndvi) <= 1
ndvi


In [ ]:
# We have NaNs because of zeros (nodata) both in nir and red arrays !
len(ndvi[np.isnan(ndvi)])

In [ ]:
# Here you can see it creates some holes in our image
fig, ax = plt.subplots(figsize=(5, 10), layout='tight')
plt.imshow(ndvi, vmin=0, vmax=0.8, cmap='viridis')  # Ignore values under 0 in the colormap

In [ ]:
# Let's create a masked array, here this function will automatically mask NaNs
masked_ndvi = np.ma.masked_invalid(ndvi)
masked_ndvi

In [ ]:
# Then we can use rasterio.fill to interpolate missing values (rasterio understands numpy masked arrays, see the docs)
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rio.fill.fillnodata(masked_ndvi), vmin=0, vmax=0.8)
fig.colorbar(ax.get_children()[0])  # Create a colorbar

In [ ]:
# Also you could just replace NaNs by zero, using np.where or other functions

## Image resampling
Now, we cant to compute a Normalized Burn Ratio.  
The formula is : **(NIR - SWIR) / (NIR + SWIR)**  
If we want to use B12 with B08, we need to upsample the array. We could also use B8A which is 20m, but we want maximum precision.  

In [ ]:
# GSD stands for Ground Sampling Distance
before.assets['B08'].extra_fields["gsd"] == before.assets['B12'].extra_fields["gsd"]

In [ ]:
# Resamplign algorithms : 
def rio_open_resampled(url, scale_factor=2):
    """Open a dataset using rasterio and scale pixel size according to given factor"""
    with rio.open(url) as ds:
        profile = ds.profile
        # compute new shape
        new_height, new_width = int(ds.height * scale_factor), int(ds.width * scale_factor)
        # read data and resample on the fly
        data = ds.read(
            out_shape=(ds.count, new_height, new_width),
            resampling=rio.enums.Resampling.bilinear
        )
        # scale image transform
        transform = ds.transform * ds.transform.scale(
            (ds.width / data.shape[-1]),
            (ds.height / data.shape[-2])
        )
        # update metadata
        profile.update({"transform": transform, "height": new_height, "width": new_width})

        return data, profile

In [ ]:
array_swir, profile = rio_open_resampled(before.assets['B12'].href, scale_factor=2)

In [ ]:
# This is a full size image, sadly we cannot use both window and out_shape when reading a dataset
rich.print(profile)

In [ ]:
aoi_window

In [ ]:
# TODO: find a way to extract ROI from array using only the aoi_window variable and numpy slicing, create a function called "windowed_array"

In [ ]:
# TODO: use numpy to concatenate the 10m result with the rest of the bands (array_before variable).

In [ ]:
# TODO: create a function that (given a Sentinel-2 STAC item) returns a 5 bands array (R,G,B,NIR,SWIR) and profile dict
# Add an optional "window" argument. Use existing functions rio_stack, rio_open_resampled and windowed_array

## Index
You need to use NIR and SWIR bands to compute the NBR for both images before and after.  
**NBR = (NIR - SWIR) / (NIR + SWIR)**  
**dNBR = NBR1 - NBR2**

In [ ]:
# TODO: create a function that compute NBR, given a 5 bands array. Use it to plot both results side by side

In [ ]:
# TODO: create a function for the whole process: given two Sentinel 2 STAC items, and an optional ROI polygon, returns the difference of NBR
# The function should take an optional window, and return a profile usable to write the result as GeoTIFF, with correct shape and dtype

In [ ]:
# TODO: Plot the dNBR result, play with colormaps and thresolds e.g. plt.imshow(dnbr>0.3)
# Try colormap "magma" or "bwr". Change vmin and vmax.
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
# TODO: use rasterio to write a GeoTIFF of your dNBR so you can download and open it in QGIS.
# To save some space and bandwith, store it with integer values (np.uint8 because GDAL can't handle np.int8).
# Ignore values under 0.

In [ ]:
profile.update({"compress": "lzw", "nodata": None})  # Use LZW compression. Unset nodata.
profile

In [ ]:
# You can use np.digitize to create a reclassified array based on thresholds
thresholds = [-0.25, -0.1, 0.1, 0.27, 0.44, 0.66]
# Source : https://un-spider.org/advisory-support/recommended-practices/recommended-practice-burn-severity/in-detail/normalized-burn-ratio

In [ ]:
reclassified_dnbr = np.digitize(dnbr, thresholds).astype("uint8")
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(reclassified_dnbr, cmap='Spectral_r')
fig.colorbar(ax.get_children()[0])

In [ ]:
# TODO: compute the Normalized Difference Water Index with the "after" image
# The formula for Sentinel-2 is:
# NDWI = (Green - NIR) / (Green + NIR)

In [ ]:
# TODO: Find a threshold in order to build a water mask. 
# Clean small blobs in mask using skimage.morphology.remove_small_objects (default params should work nicely)
# Apply this mask to plot the dNBR data without in-water pixels

A new index for Sentinel-2 (NBR+) was [recently published](https://www.researchgate.net/publication/359705541_Normalized_Burn_Ratio_Plus_NBR_A_New_Index_for_Sentinel-2_Imagery).  
The worklow is the same, but the NBR+ formulae is different. It may give better result (and avoid detecting water bodies as fires).  
**NBR+ = (B12 − B8 − B3 − B2) / (B12 + B8 + B3 + B2)**

We used B8, but you could also try with B8A and see the difference (should not be significative, it depends on the type of vegetation though).

In [ ]:
# Bonus TODO: Adapt your previous "NBR from array" function, it should accept an optional argument so we can compute either dNBR or dNBR+.
# Use it to compare the results in leafmap or QGIS. Do you think it is better or worse ? Use a visual image and / or false color composite (after fire) as basemap.

## Vectorize raster blobs

In [ ]:
geoms, values = [], []
# We create a list of shapes using source image transform
for geom, value in rio.features.shapes(
    reclassified_dnbr,
    transform=profile['transform'],
    mask=(reclassified_dnbr >= 3),  # also we can mask some values
):
    geoms.append(shapely.geometry.shape(geom))
    values.append(value)

In [ ]:
# Create pandas series with our data lists: polygons and pixel values
geoms = gpd.GeoSeries(geoms)
values = pd.Series(values)

In [ ]:
# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(values.rename("dNBR_class"), geometry=geoms, crs=crs)
gdf["area_ha"] = gdf.area / 10_000
len(gdf)

In [ ]:
# We have a lot of isloated pixels. It's better to remove them or leafmap will take too long to render
gdf.sort_values("area_ha").head()
gdf = gdf[gdf.area_ha >= 0.1]
len(gdf)
# NB: be careful of what you vectorize, many isloated pixel would result in a huge list of polygons
# If you ever need to vectorize binary objects (e.g. the result of a threshold), you should always use binary cleaning before converting to polygons.

In [ ]:
gdf.explore("dNBR_class", tiles=folium_tile, cmap="copper_r", style_kwds={"stroke": None})

In [ ]:
# TODO: create a vector layer of the water mask you just built: it will be just a list of shapes since the pixel value is not useful
# Add it to a leafmap with a visual rendering of the source image. Check if it fits all right, or may be you need to adjust threshold / binary cleaning parameters ?

## Rasterize vector shapes
Sometimes, you'll need to manipulate vector shapes as numpy arrays (e.g. ROI extraction, ground truth labelling, ...)

In [ ]:
# Let's download some cadastral data from IGN WFS
wfs_parcelles = "https://wxs.ign.fr/parcellaire/geoportail/wfs?SERVICE=WFS&VERSION=2.0.0&REQUEST=GetFeature&OUTPUTFORMAT=application/json"
wfs_parcelles += "&TYPENAME=CADASTRALPARCELS.PARCELLAIRE_EXPRESS:parcelle&COUNT=1000&CQL_FILTER=code_insee=33529&SRSNAME=" + str(crs)
parcelles = None
i = 0
# Since a WFS query with this public URL is limited to 1000 results, we need to do it in several times
while True:
    result = gpd.read_file(wfs_parcelles + f"&startIndex={i}")
    if len(result) == 0:
        # In that case the previous loop was the last with data
        break
    if parcelles is None:
        # First iteration
        parcelles = result
    else:
        parcelles = pd.concat([parcelles, result])

    i += 1000

In [ ]:
parcelles.reset_index(inplace=True, drop=True)
parcelles.head()

In [ ]:
parcelles.plot("id", categorical=True, figsize=(5,8))
plt.axis("off")

In [ ]:
# For the next step, we need a unique integer ID that does not start at zero
parcelles["raster_id"] = parcelles.index + 1
parcelles["raster_id"].max()

In [ ]:
# To speed up the process, let's ignore small parcels (under 1 ha)
parcelles_1ha = parcelles.loc[parcelles.area > 10_000]
len(parcelles_1ha)

In [ ]:
parcelles_1ha.plot()
plt.axis("off")

In [ ]:
# We need to iterate over a list of pairs (geometry, value): the pixel value in target array
iter_shapes = ( (feat.geometry, feat.raster_id) for name, feat in parcelles_1ha.iterrows() )
# Here, name is the default df index value. We don't use a list comprehension on purpose.

In [ ]:
# A generator is an object that allow you not to load all the dataset at once (your RAM will love them)
parcelles.iterrows(), iter_shapes

In [ ]:
# Now let's rasterize, we need a target shape and image transform, 0 is for values outside polygons
parcelles_array = rio.features.rasterize(iter_shapes, nbr_after.shape, fill=0, transform=profile['transform'], dtype="uint16")

In [ ]:
# We lost some shapes in the process : some are too far north of our ROI
len(np.unique(parcelles_array)), parcelles_array.max()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(np.ma.masked_equal(parcelles_array, 0), cmap='prism', interpolation="nearest")
plt.axis("off")

## Zonal statistics
You'll often need to compute statistics using vector data. Also, one common method for image classification is [OBIA](https://en.wikipedia.org/wiki/Image_analysis#Object-based).  
We won't cover this here, but the idea is that you consider group of pixels, in opposition to pixel classification. It generally implies segmentation algorithms.  
Cadastral data is also very used in that kind of studies. A dataframe of polygons with many attributes (features) can then feed any kind of machine learning pipeline.

There are many ways to compute zonal statistics. Here we can use scikit-image to do it all with arrays.  
Most zonal stats tools are operating that way behind the curtains.  

In [ ]:
# For available properties, see https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops
# There a lot of useful geometrical functions already defined, like area or centroid.
props = ["label", "area", "intensity_min", "intensity_max", "intensity_mean"]
result = skimage.measure.regionprops_table(parcelles_array, intensity_image=dnbr, properties=props)
result

In [ ]:
df = pd.DataFrame(result).set_index('label')
df.head()

In [ ]:
# Now you can use Pandas to plot or compute statistics
parcelles_stats = parcelles_1ha.set_index("raster_id").join(df)
parcelles_stats.plot("intensity_mean", figsize=(10,10))
plt.axis("off")

In [ ]:
# See here how you can define your own numpy functions for regionprops
def zonal_stats(
    labels: np.ndarray,
    intensity: np.ndarray = None,
    props: tuple = ("label", "area"),
    extra_props: tuple = (),
    numerical: bool = True,
) -> pd.DataFrame:
    """Use skimage regionprops with custom stats"""
    if numerical and intensity is not None:

        def min(regionmask, intensity):  # pylint: disable=redefined-builtin
            return np.amin(intensity[regionmask])

        def max(regionmask, intensity):  # pylint: disable=redefined-builtin
            return np.amax(intensity[regionmask])

        def mean(regionmask, intensity):
            return np.mean(intensity[regionmask])

        def std(regionmask, intensity):
            return np.std(intensity[regionmask])

        def quartiles(regionmask, intensity):
            return np.percentile(intensity[regionmask], q=(25, 50, 75))

        # Here, region mask is an index of pixels within a labelled shape
        # You could also add a custom mask / NaNs base on intensity values if needed
        # Also sometime you may need to pass an intensity array with NaNs, so here you could replace max by nanmax

        extra_props = extra_props + (min, max, mean, std, quartiles)

    # If intensity is multiband, you'll have stat column for each band named like "max-3"
    table = skimage.measure.regionprops_table(
        labels, intensity, props, extra_properties=extra_props
    )
    props_df = pd.DataFrame(table)
    props_df.set_index("label", inplace=True)
    return props_df

In [ ]:
df = zonal_stats(parcelles_array, nbr_before)
df.head()

### Notes about zonal stats :
Since you can pass custom functions to regionprops using "extra_properties", you can do a lot of things. But it can be very slow if you're dealing with full size, high resolution images.  
One other popular package for zonal stats and point queries is rasterstats (not installed here), it is really handy and also allow custom functions.  
But, if you're dealing with big time series 4-dimensional arrays, you'll need the xarray-spatial package which has a ton of functions and will handle image resampling for you.

## Going further
If you have time, you can now proceed with a full image analysis. You may have noticed there are at least 2 other burnt areas in the same Sentinel-2 tile.  

In [ ]:
# Bonus TODO: choose another "after" date so you can study all fires in the S2 tile. Compute dNBR.
# Isolate these fires as 3 new ROI / commune clusters. Create subplots of reclassified dNBR, with 3 fires side by side.
# Use the "communes" gdf to measure the % of burnt area in vector mode. We'll consider burnt anything with dNBR >= 0.1
# Create functions, nobody wants to read a huge monolithic code block.

In [ ]:
# Bonus TODO: find a way to distinct forest areas, with any kind of dataset (vector or raster, from IGN WFS, Planetary Computer, etc.).
# Compute surfaces in raster mode using zonal stats, and binary rasterized shapes (sum/area)
# We want to map the surface of burnt forest, absolute (you could then use marker size) and relative (for choropleth map).

In [ ]:
# If you're already done, may be you'll want to see some PC notebooks and learn how to deal with 4D xarrays and time series.
# N.B: if you don't want to use xarray, you could still obtain the same result, processing dates one by one with rasterio.
# Xarray is useful for huge datasets, and to scale with dask. Not always. Learn to use the right tool for the job.
# See these notebooks "quickstarts/reading-stac", "tutorials/cloudless-mosaic-sentinel2" and "tutorials/zonal_statistics".

# Bonus+ TODO: compute NDVI time serie of our area_of_interest, and find out how resilient the vegetation is after the fire.
# Create plots of NDVI trends at a parcel level.